# Introduction of PyTest Framework

#### What is pytest?

- Pytest is a unit testing framework
    - Allows us to create:
        1. Tests
        2. Test modules
        3. Test fixtures

- Uses the `assert` statement
    - Makes it more simple than other frameworks
    
- Has command line parameters to decide the tests to run, and in which order

- When we create a test for pytest, we always have `test_` before the function name
    - E.g. `test_multiplier`
    
- Similar tests can be grouped together by having them all in the same module (i.e. the same .py script file) or class

____


# Test Discovery Function

**What does "test discovery" mean?**

- If you use the right naming conventions, pytest will recognize the tests automatically and can execute them
    - If we create a test function, we already noted that its name should begin with `test_`
    - If we create a test class, its name should begin with `Test` and should not haev an `init` method
    - The filename for the script containing the tests should begin with `test_`
        - E.g. `test_example.py`

_____

# XUnit Style Setup and Teardown

- XUnit style setup/teardown functions execute code before and after:
    1. Test modules
    2. Test functions
    3. Test classes
    4. Test methods inside test classes
    
- The point is to reduce duplicated code

*Example - for functions*

```python
def setup_function(function):
    if function == test_1:
        print("\nSetting up test_1!")
    elif function == test_2:
        print("\nSetting up test_2!")
    else:
        print("\nSetting up unknown test!")

def teardown_function(function):
    if function == test_1:
        print("\nTearing down test_1!")
    elif function == test_2:
        print("\nTearing down up test_2!")
    else:
        print("\nTearing down up unknown test!")
```

- pytest will automatically recognize the setup and teardown functions and run them during the testing functions

*Example - for modules*

```python
def setup_module():
    print('Setting up module!')

def teardown_module():
    print('Tearing down module!')
```

- Gonna skip the example for classes

____

# Test Fixtures Environment

- Test fixtures are like the setup and teardown functions in that they help reduce duplicated code
- Fixtures are applied to functions using a decorator

*Example*

```python
import pytest

@pytest.fixture()
def setup():
    print("\nSetup")

def test_1():
    print("Executing test1")
    assert True
    
def test_2():
    print("Executing test1")
    assert True
```

- If we run this code, the setup fixture **won't run**
    - We need to add it as a parameter to the tests
    
```python
def test_1(setup):
    print("Executing test1")
    assert True
    
def test_2():
    print("Executing test1")
    assert True
```

- Now, when we run this, the setup will run before `test_1`, but not `test_2`
    - We can specify the fixture in another way for `test_2`
    
```python
def test_1(setup):
    print("Executing test1")
    assert True
    
@pytest.mark.usefixtures("setup")
def test_2():
    print("Executing test1")
    assert True
```

- Now, the setup fixture will run for both
    - The advantage of calling it using the second method is that we can easily specify different fixtures for different test functions

- If we wanted the fixture to run for each test, we can change our code to the following:

```python
import pytest

@pytest.fixture(autouse=True)
def setup():
    print("\nSetup")

def test_1():
    print("Executing test1")
    assert True
    
def test_2():
    print("Executing test1")
    assert True
```

- Each fixture can have its own setup and teardown code
    - The teardown can be specified using:
        1. The `yield` keyword
        2. The request-context object's `addfinalizer` method

```python
@pytest.fixture()
def setup():
    print("\nSetup")
    yield
    print("Teardown!")
```

- In this snippet, everything after the `yield` runs after the test is completed

```python
@pytest.fixture()
def setup():
    print("\nSetup")
    def teardown():
        print("Teardown!")
    request.addfinalizer(teardown)       
```

- This snippet will do exactly the same thing as the one above
    - It's a bit more complicated, but is also capable of more
        - E.g. can specify multiple teardown functions

#### Test fixtures scope

- Test fixtures can be called in four ways:
    1. Function
        - Runs the fixture **once for each test**
    2. Class
        - Runs the fixture **once for each class of tests**
    3. Module
        - Runs the fixture **once for each module**
    4. Session
        - Runs the fixture **once pytest starts**

*Example*

```python
import pytest

@pytest.fixture(scope='session', autouse=True)
def setup_session():
    print('Seting up session')
    
@pytest.fixture(scope='module', autouse=True)
def setup_module():
    print('Seting up module')
    
@pytest.fixture(scope='function', autouse=True)
def setup_function():
    print('Seting up function')
    
def test_1():
    print('Running test_1')
    assert True

def test_2():
    print('Running test_2')
    assert True
```

- If this is our test script, when we run it, it'll run the session fixture first, the module fixture next, then run the function fixture twice (once for each test)

#### Test fixture return objects and parameters

- We can get data back from our fixtures to be used in the tests

*Example*

```python
import pytest

@pytest.fixture(params=[1,2,3])
def setup(request):
    return_val = request.param
    print('Setup! return_val = {}'.format(return_val))
    return return_val
    
def test_1(setup):
    print('setup = {}'.format(setup))

```

- When we run this code, **the test will run three times**
    - Once for each parameter defined in the fixture
        - The test output will be:
        
```
test_file.py::test_1[1]
Setup! return_val = 1

setup = 1
PASSED

test_file.py::test_1[2]
Setup! return_val = 2

setup = 2
PASSED

test_file.py::test_1[3]
Setup! return_val = 3

setup = 3
PASSED

======== 3 passed in 0.01 seconds ========

```

- **Note**: we need to be careful when using this approach
    - If the result should be the same for the different parameters then it should be fine
        - But if the results are different, we should be defining distinct test functions

____

# Assert Statements and Exceptions

- pytest uses the built-in `assert` statement to perform the tests
    - Examples
        - `assert a == 1`
        - `assert a <= 1`
        - `assert a > 1`
        - `assert a != 1`
        
- When a test fails, pytest returns a message to give more detail

#### Comparing floatin point values

- Let's say we have the statement `assert 0.33333 == 1/3`
    - This will fail, even though the values are *effectively equal*
    
- We can solve this issue by using the pytest `approx` function
    - E.g. `assert 0.33333 == approx(1/3)`
        - This is the same as `np.isclose`

#### Verifying exceptions

- In some cases, we want to test that the correct expection is thrown under certain circumstances

*Example*

- We define a function that we know will raise an error

```python
def cause_error():
    assert 1 == 2
```

- We know this function will raise an assertion error
    - Now, we write a test to confirm this
    
```python
def test_cause_error():
    with raises(AssertionError):
        cause_error()
```

- Since the function raises an AssertionError, the test will pass